# Regressions and Causal Inference

## Comparison of Two Means

One sample is $\{y^{(0)}_{1}, \ldots, y^{(0)}_{n_1}\} \sim (\mu_0, \sigma^2_0)$. The other sample is $\{y^{(1)}_{1}, \ldots, y^{(1)}_{n_0}\}\sim (\mu_1, \sigma^2_1)$.

Compute $\bar{y}^(0) = mean(y^{0}_i)$ and similarly $\bar{y}^(1) = mean(y^{1}_i)$. Let the difference be $\hat{\Delta} = \bar{y}^{1} - \bar{y}^0$.

**Exercise**: Compute the mean and the variance of $\hat{\Delta}$.

In [8]:
from scipy import stats
import numpy as np

# Generate example data
np.random.seed(20250109)
y0 = np.random.normal(loc=10, scale=2, size=30)  # First sample
y1 = np.random.normal(loc=10, scale=2, size=30)  # Second sample

# Calculate means
mean0 = np.mean(y0)
mean1 = np.mean(y1)


print(f"\nMean of first sample: {mean0:.4f}")
print(f"Mean of second sample: {mean1:.4f}")
print(f"Difference in means: {mean1 - mean0:.4f}")



Mean of first sample: 9.8463
Mean of second sample: 10.2301
Difference in means: 0.3838


In [9]:
# Perform independent t-test
t_stat, p_value = stats.ttest_ind(y0, y1)

print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")


T-statistic: -0.7400
P-value: 0.4623


The difference in mean can be equivalently written as $y^{(0)}_i = \mu_1 + \epsilon^{(0)}_i$ and $y^{(1)}_i = \mu_1 + \epsilon^{(1)}_i$. More concisely, we can combine the two groups into one regression
$$
y_i = \mu_0 + D_i \Delta + e_i,
$$ 
with total sample size $n = n_1 + n_2$, where $D_i$ is the indicator function for the group $\{(1)\}$ and $e_i = D_i \epsilon^{(1)}_i + (1-D_i)\epsilon^{(0)}_i$.

In [10]:
import statsmodels.api as sm

# Combine the samples
y = np.concatenate([y0, y1])
D = np.concatenate([np.zeros(len(y0)), np.ones(len(y1))])

# Add a constant term for the intercept
X = sm.add_constant(D)

# Run the OLS regression
model = sm.OLS(y, X).fit()

# Print the summary of the regression
print(model.summary())

# Obtain the t-statistic for the interaction term
t_stat_interaction = model.tvalues[1]
print(f"T-statistic for the interaction term: {t_stat_interaction:.4f}")

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.5476
Date:                Wed, 08 Jan 2025   Prob (F-statistic):              0.462
Time:                        05:35:16   Log-Likelihood:                -125.96
No. Observations:                  60   AIC:                             255.9
Df Residuals:                      58   BIC:                             260.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.8463      0.367     26.851      0.0

The above computation applies to any two observable samples.

## Causation

* Causality is philosophical.
* [Aristotle's four causes](https://en.wikipedia.org/wiki/Four_causes): various forms
* [Hume's causation](https://www.britannica.com/topic/causation): simultaneity or connection
* [Occam’s razor](https://www.britannica.com/topic/Occams-razor): the simpler is preferred

## Randomized Control Trials

* The **potential outcome framework**: $(y_i^{(1)}, y_i^{(0)})$.
* Only one outcome is observable: $y_i = D_i y_i^{(1)} + (1-D_i) y_i^{(0)}$.
* **Individual treatment effect**  $\Delta_i = y_i^{(1)} - y_i^{(0)}$ is infeasible.

* **Average treatment effect** $ATE = E[ \Delta_i]$.
* **Average treatment effect on the treated** $ATE = E[ \Delta_i | D_i = 1]$.

* Key assumption: $(y_i^{(1)}, y_i^{(0)}) \perp D_i$.
* It enables the identification of ATE
\begin{align*}
ATE & \equiv E[y_i^{(1)} -  y_i^{(0)}] \\
    & \stackrel{ind.}{=} E[y_i^{(1)} -  y_i^{(0)} | D_i] \\
    & = E[y_i^{(1)} | D_i ] - E[y_i^{(0)} | D_i] \\
    & = E[y_i^{(1)} | D_i = 1 ] - E[y_i^{(0)} | D_i = 0] 
\end{align*}
as the two terms on the RHS are feasible


* ATE and ATET are identical
$$
ATE = E [\Delta_i ] = E [\Delta_i | D_i  ] = E [\Delta_i | D_i = 1 ] = ATET 
$$

Economic applications: microfinance, deworming, eye glasses, etc.

## Conditional ATE and ATET

* Add contorl variables $X_i$. 
* **Conditional average treatment effect (CATE)** $ATE(X_i) = E[ \Delta_i | X_i]$.
* **Conditional average treatment effect on the treated (CATET)** $ATET(X_i) = E[ \Delta_i | X_i, D_i = 1]$.

**Unconfoundedness** (also known as **conditional independence assumption**): $(y_i^{(1)}, y_i^{(0)}) \perp D_i | X_i$.
$$
ATE(X_i) = E[y_i^{(1)} | X_i, D_i = 1 ] - E[y_i^{(0)} | X_i, D_i = 0]
$$
and similarly $ATE(X_i) = ATET(X_i)$

Both $E[y_i^{(1)} | X_i, D_i = 1 ]$ and $E[y_i^{(0)} | X_i, D_i = 0]$ are identifiable from observed data.

### Regression

The conditional mean is a nonlinear function in general.

Now, we impose the **linear assumption**:

* In conditional mean form: 
$$
E[y_i^{(k)} | X_i, D_i = k ] = \mu_k + \beta_k X_i, \textrm{ where } k \in \{0,1\}.
$$
* In regression form:
$$
y_i^{(k)} = \mu_k + \beta_k X_i + \epsilon_i^{(k)}
$$
where $E[ \epsilon_i^{(k)} | X_i, D_k = k]=0$.

The CATE is
$$
ATE(X_i) = (\mu_1 - \mu_0) + (\beta_1 - \beta_0)X_i = \Delta_{\mu} + \Delta_{\beta} X_i
$$

The above ATE can be equivalently find in the linear regression
$$
y_i = \mu_0 + \Delta_{\mu} D_i + \beta_0 + \Delta_{\beta} D_i X_i + e_i =  W_i'\theta + e_i
$$
where $e_i = D_i \epsilon_i^{(1)} + (1-D_i) \epsilon_i^{(2)}$, the vector $W_i$ collects all regressors and $\theta$ collects all parameters. 

* CIA implies mean independence $0 = E[e_i | X_i, D_i ] = E[e_i | W_i]$. 
* Under **mean independence**, OLS is an unbiased estimator of $\theta$.
* All the above analysis is conducted under a finite $n$. 


## Estimation

* Estimation without covariates is the same as the statistical comparison of the two means.
* Estimation with covariates is most conveniently implemented via a linear regression with interactive terms.

* Testing the ATE of the interference $D_i$ is equivalent to estimating the joint null hypothesis $H_0: \Delta_{\mu} = \Delta_{\beta} = 0$.  

* Causality is a narrative behind the statistical procedure.

In [11]:
# Generate a normally distributed variable x
x = np.random.normal(loc=5, scale=1, size=len(y))

# Combine the samples and the new variable
W_with_x = np.column_stack((D, x, D * x))

# Add a constant term for the intercept
W_with_x = sm.add_constant(W_with_x)

# Run the OLS regression
model_with_x = sm.OLS(y, W_with_x).fit()

# Print the summary of the regression
print(model_with_x.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                 -0.026
Method:                 Least Squares   F-statistic:                    0.5044
Date:                Wed, 08 Jan 2025   Prob (F-statistic):              0.681
Time:                        05:35:16   Log-Likelihood:                -125.44
No. Observations:                  60   AIC:                             258.9
Df Residuals:                      56   BIC:                             267.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.8847      2.275      3.466      0.0

In [12]:
# Define the restriction matrix for the Wald test
# We want to test if the coefficients of D and D*x are jointly zero
restriction_matrix = np.zeros((2, len(model_with_x.params)))
restriction_matrix[0, 1] = 1  # Coefficient of D
restriction_matrix[1, 3] = 1  # Coefficient of D*x

# Perform the Wald test
wald_test = model_with_x.wald_test(restriction_matrix)

# Print the results of the Wald test
print(wald_test)

<F test: F=array([[0.72954896]]), p=0.4866517134927578, df_denom=56, df_num=2>


C:\Users\zhent\AppData\Roaming\Python\Python311\site-packages\statsmodels\base\model.py:1914: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


## Simplifying Assumptions

* Homogenous slope coefficients: $\beta_0 = \beta_1$. No interactive term is needed.

* Extending the analysis from a binary $D_i$ to a continuous $D_i$ justifies the usual OLS practice. 

## Alternative Formulation


* Notice $y_i^{(1)} = D_i y_i$ and $y_i^{(0)} = (1-D_i) y_i$


\begin{align*}
ATE & = E[y_i^{(1)} | D_i = 1 ] - E[y_i^{(0)} | D_i = 0] \\
    & = \frac{E[D_i y_i ]}{\Pr(D_i = 1)} - \frac{E[(1-D_i) y_i]}{\Pr(D_i = 0)}
\end{align*}